<a href="https://colab.research.google.com/github/yvrjsharma/HugginFace_Gradio/blob/main/DDColor_Gradio_Demo_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/piddnad/DDColor.git

In [2]:
%cd DDColor

/content/DDColor


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install gradio -q

In [ ]:
!pip install timm gradio_imageslider -q

In [ ]:
!pip install modelscope -q

In [6]:
from modelscope.hub.snapshot_download import snapshot_download

model_dir = snapshot_download('damo/cv_ddcolor_image-colorization', cache_dir='./modelscope')
print('model assets saved to %s'%model_dir)

2024-01-19 03:47:46,140 - modelscope - INFO - PyTorch version 2.1.0+cu121 Found.
2024-01-19 03:47:46,144 - modelscope - INFO - TensorFlow version 2.15.0 Found.
2024-01-19 03:47:46,145 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-01-19 03:47:46,147 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-01-19 03:47:46,210 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 d1bf70dc840bbc78c6b639e013c9762e and a total number of 953 components indexed
2024-01-19 03:47:50,058 - modelscope - WARNING - Model revision not specified, use revision: v1.02
Downloading: 100%|██████████| 1.39k/1.39k [00:00<00:00, 6.37MB/s]
Downloading: 100%|██████████| 235k/235k [00:00<00:00, 7.11MB/s]
Downloading: 100%|██████████| 199k/199k [00:00<00:00, 6.00MB/s]
Downloading: 100%|██████████| 94.9k/94.9k [00:00<00:00, 5.95MB/s]
Downloading: 100%|██████████| 117k/117k [0

model assets saved to ./modelscope/damo/cv_ddcolor_image-colorization


In [11]:
import sys
sys.path.append('/DDColor')

import argparse
import cv2
import numpy as np
import os
from tqdm import tqdm
import torch
from basicsr.archs.ddcolor_arch import DDColor
import torch.nn.functional as F

import gradio as gr
from gradio_imageslider import ImageSlider
import uuid
from PIL import Image

model_path = 'modelscope/damo/cv_ddcolor_image-colorization/pytorch_model.pt'
input_size = 512
model_size = 'large'


# Create Image Colorization Pipeline
class ImageColorizationPipeline(object):

    def __init__(self, model_path, input_size=256, model_size='large'):

        self.input_size = input_size
        if torch.cuda.is_available():
            self.device = torch.device('cuda')
        else:
            self.device = torch.device('cpu')

        if model_size == 'tiny':
            self.encoder_name = 'convnext-t'
        else:
            self.encoder_name = 'convnext-l'

        self.decoder_type = "MultiScaleColorDecoder"

        if self.decoder_type == 'MultiScaleColorDecoder':
            self.model = DDColor(
                encoder_name=self.encoder_name,
                decoder_name='MultiScaleColorDecoder',
                input_size=[self.input_size, self.input_size],
                num_output_channels=2,
                last_norm='Spectral',
                do_normalize=False,
                num_queries=100,
                num_scales=3,
                dec_layers=9,
            ).to(self.device)
        else:
            self.model = DDColor(
                encoder_name=self.encoder_name,
                decoder_name='SingleColorDecoder',
                input_size=[self.input_size, self.input_size],
                num_output_channels=2,
                last_norm='Spectral',
                do_normalize=False,
                num_queries=256,
            ).to(self.device)

        self.model.load_state_dict(
            torch.load(model_path, map_location=torch.device('cpu'))['params'],
            strict=False)
        self.model.eval()

    @torch.no_grad()
    def process(self, img):
        self.height, self.width = img.shape[:2]
        img = (img / 255.0).astype(np.float32)
        orig_l = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:, :, :1]  # (h, w, 1)

        # resize rgb image -> lab -> get grey -> rgb
        img = cv2.resize(img, (self.input_size, self.input_size))
        img_l = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)[:, :, :1]
        img_gray_lab = np.concatenate((img_l, np.zeros_like(img_l), np.zeros_like(img_l)), axis=-1)
        img_gray_rgb = cv2.cvtColor(img_gray_lab, cv2.COLOR_LAB2RGB)

        tensor_gray_rgb = torch.from_numpy(img_gray_rgb.transpose((2, 0, 1))).float().unsqueeze(0).to(self.device)
        output_ab = self.model(tensor_gray_rgb).cpu()  # (1, 2, self.height, self.width)

        # resize ab -> concat original l -> rgb
        output_ab_resize = F.interpolate(output_ab, size=(self.height, self.width))[0].float().numpy().transpose(1, 2, 0)
        output_lab = np.concatenate((orig_l, output_ab_resize), axis=-1)
        output_bgr = cv2.cvtColor(output_lab, cv2.COLOR_LAB2BGR)

        output_img = (output_bgr * 255.0).round().astype(np.uint8)

        return output_img


# Initialize
colorizer = ImageColorizationPipeline(model_path=model_path,
                                      input_size=input_size,
                                      model_size=model_size)


# Create inference function for gradio app
def colorize(img):
  image_out = colorizer.process(img)
  # Generate a unique filename using UUID
  unique_imgfilename = str(uuid.uuid4()) + '.png'
  cv2.imwrite(unique_imgfilename, image_out)
  return (img, unique_imgfilename)


# Gradio demo using the Image-Slider custom component
with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column():
      bw_image = gr.Image(label='Black and White Input Image')
      btn = gr.Button('Convert using DDColor')
    with gr.Column():
      col_image_slider = ImageSlider(position=0.5,
                                     label='Colored Image with Slider-view')

  btn.click(colorize, bw_image, col_image_slider)
demo.launch()